# Ethereum Contracts

This notebook demonstrates how to use the `ethereum_contracts` dataset to answer questions about the history of contracts on Ethereum.

Additional information about this dataset can be found [here](https://github.com/paradigmxyz/paradigm-data-portal/tree/main/datasets/ethereum_contracts)

The examples below use [polars](https://github.com/pola-rs/polars) library to read the files directly without needing to import them to a database. Polars is also used in streaming mode, so that the dataset does not need to fit into memory.


## Ethereum contracts by the numbers

All of these fun facts are computed in the examples below:
```
- 20% of all ethereum contracts have been deployed by just 10 EOAs
- 49% of all ethereum contracts have been created by just 10 factories
- 56% of all ethereum contracts use just 10 bytecode templates
- 0.46% of ethereum addresses have been deployed more than once
- The most times that a single contract address has been deployed is 848
- The most contracts deployed in a single transaction or block is 701
- 57M contracts have been deployed by 4.2M EOAs using 492K factories
- Only 878K unique contracts have been deployed
- The most leading zeros in a contract address is 14, of which there are 2 contracts
```

In [1]:
import math
import os
import shutil
import subprocess

import ctc
import ctc.rpc
from ctc.toolbox import pl_utils
import polars as pl
import matplotlib.pyplot as plt
import toolplot
import numpy as np
import toolstr

In [2]:
# set up env

pl_utils.set_column_display_width()

data_path = '~/pdp/ethereum_contracts/ethereum_contracts__v1_0_0__*.parquet'
data_path = os.path.expanduser(data_path)

# Let's count the entites and events in the data...

In [3]:
def get_unique_column_count(column):
    result = pl.scan_parquet(data_path).select(
        pl.col(column).n_unique(),
    ).collect(streaming=True)
    return result[column][0]

### How many contract deployments have there been?

In [4]:
result = pl.scan_parquet(data_path).select(pl.col('create_index').count()).collect()
n_deployments = result['create_index'][0]

n_deployments

57887821

### How many unique contract addresses have been deployed?

In [5]:
n_unique_contract_addresses = get_unique_column_count('contract_address')

n_unique_contract_addresses

57314368

### How many factories have been used to create contracts?

In [6]:
get_unique_column_count('factory')

491663

### How many EOAs have deployed contracts?

In [7]:
get_unique_column_count('deployer')

4228306

### How many unique contract bytecodes have been deployed?

In [8]:
get_unique_column_count('code_hash')

878235

### How many contract addresses have been deployed more than one time?

In [9]:
n_deployed_more_than_once = (
    pl.scan_parquet(data_path)
    .groupby(pl.col("contract_address"))
    .agg(pl.count())
    .filter(pl.col('count') > 1)
    .select(pl.count())
    .collect(streaming=True)
).rows()[0][0]

n_deployed_more_than_once

PARTITIONED DS: estimated cardinality: 1 exceeded the boundary: 0.4, running default HASH AGGREGATION


265431

In [10]:
more_than_once_percent = toolstr.format(
    n_deployed_more_than_once / n_unique_contract_addresses,
    percentage=True,
    decimals=2,
)
print(more_than_once_percent, 'of contract addresses have been deployed more than once')

0.46% of contract addresses have been deployed more than once


# Let's look at some top 10 leaderboards

In [11]:
def get_top_n(column, n):
    col = '0x' + (pl.col(column).struct.field(column).bin.encode('hex'))
    return (
        pl.scan_parquet(data_path)
        .select(pl.col(column).value_counts())
        .select([col.alias(column), pl.col(column).struct.field('counts')])
        .sort('counts', descending=True)
        .head(n)
        .collect(streaming=True)
    )

### Which EOAs have deployed the most contracts?

In [12]:
top_10_deployers = get_top_n('deployer', 10)

top_10_deployers

deployer,counts
str,u32
"""0xffff46e05a09314daae9176fc32dba0f4172dcdb""",2854163
"""0xe5c99b740572c2dabf7e3a418c4e9df2f793a599""",1822822
"""0x6cace0528324a8afc2b157ceba3cdd2a27c4e21f""",1579922
"""0x5f5a901463fbffeca2b4dfad613bc49e445467cd""",954224
"""0xf24c621e5108607f4ec60e9c4f91719a76c7b3c9""",952859
"""0x9901c66f2d4b95f7074b553da78084d708beca70""",801294
"""0xfc9161b4ca1e5db6fecf7d914a73c95ac969b7b9""",665752
"""0x2aab3ddd93ededc4d4275bef3e4f4d2ca95e0008""",600176
"""0x06ca51ab3c82c4a834872ebff806a9a78e2aea47""",542637


In [13]:
top_10_deployer_percent = toolstr.format(
    top_10_deployers['counts'].sum() / n_deployments,
    percentage=True,
    decimals=1,
)

print('The top 10 deployers have deployed', top_10_deployer_percent, 'of all contracts on Ethereum')

The top 10 deployers have deployed 19.5% of all contracts on Ethereum


### Which factories have deployed the most contracts?

In [14]:
top_10_factories = get_top_n('factory', 10)

top_10_factories

factory,counts
str,u32
"""0x0000000000004946c0e9f43f4dee607b0ef1fa1c""",10364776
"""0x0000000000b3f879cb30fe243b4dfee438691c04""",6493824
"""0x0de8bf93da2f7eecb3d9169422413a9bef4ef628""",2324361
"""0x0000000000007f150bd6f54c40a34d7c3d5e9f56""",1647068
"""0xa3c1e324ca1ce40db73ed6026c4a177f099b5770""",1586334
"""0xa5409ec958c83c3f309868babaca7c86dcb077c1""",1552418
"""0x8a91c9a16cd62693649d80afa85a09dbbdcb8508""",1088378
"""0x0a252663dbcc0b073063d6420a40319e438cfa59""",1021675
"""0x881d4032abe4188e2237efcd27ab435e81fc6bb1""",994275


In [15]:
top_10_factory_percent = toolstr.format(
    top_10_factories['counts'].sum() / n_deployments,
    percentage=True,
    decimals=1,
)

print('The top 10 factories have deployed', top_10_factory_percent, 'of all contracts on Ethereum')

The top 10 factories have deployed 48.4% of all contracts on Ethereum


### Which contract bytecodes are most common?

In [16]:
top_10_bytecodes = get_top_n("code_hash", 10)

top_10_bytecodes

code_hash,counts
str,u32
"""0x1d93f60f105899172f7255c030301c3af4564edd4a48577dbdc448aec7ddb0ac""",10365216
"""0xd80cd839dd3957d572b90780ada202a13936fa2875daea94216263371e9ef1d2""",6494337
"""0xc5d2460186f7233c927e7db2dcc703c0e500b653ca82273b7bfad8045d85a470""",4853969
"""0x562d59a51820d47f520c975e0b2bcffac644a509749a3161f481f57b6e826d21""",2324361
"""0x8f789d24f75f80df63f1d69c18963fb4c2dea81ade3b780fd0c886e4d1fabc4a""",1647068
"""0xce33220d5c7f0d09d75ceff76c05863c5e7d6e801c70dfe7d5d45d4c44e80654""",1595657
"""0x99c99f1a6d65a9097e0f8ca61683878ed26099e347359f5cdc600bd194dbe908""",1548789
"""0x647d5d7eb5af6fbf9ed95320cadf658cec3819ffa536adc7b6387fdf1b72123d""",1542104
"""0x2df39c0062882ba634fb24c89e7f9871ecef10a6e6a6e79a336f6ed463d5c942""",1175582


In [17]:
top_10_bytecode_percent = toolstr.format(
    top_10_bytecodes['counts'].sum() / n_deployments,
    percentage=True,
    decimals=1,
)

print('The top 10 bytecodes represent', top_10_bytecode_percent, 'of all contracts on Ethereum')

The top 10 bytecodes represent 56.4% of all contracts on Ethereum


### Which contract addresses have been deployed the most number of times?

In [18]:
top_10_contract_addresses = get_top_n('contract_address', 10)

top_10_contract_addresses

contract_address,counts
str,u32
"""0xba4ac7aadfa00003a20c954e077d5c81994c8ece""",848
"""0x9c14d5cb81d576e97874b593bffdee670347bfd6""",843
"""0xa46ec029df976770a75b2ce42fd4276ddc26a961""",818
"""0x08bd3b769ffd7992e1d14101a2235eeff7232c57""",817
"""0x3e2d8eabd99bc994d2ab4a804c6f804120af123e""",743
"""0x71664ea68d994fb064aed08807257b2c6dfbd65d""",730
"""0x472875a6fde6cc05b53e58f7e7064d5add45c4bf""",729
"""0x8a454242940937c95dffb7b038fe27b61817f036""",497
"""0x19486d3a1da8cfeb4cd19c9c4fe5624a6587673c""",496


### What is the most contracts deployed in a single transaction?

In [19]:
top_10_transactions = get_top_n('transaction_hash', 10)

top_10_transactions

transaction_hash,counts
str,u32
"""0x81ab1942007f36adf8a8b5930e1b0e5e9f9880dca65e972a43b13be51cfe4ad4""",701
"""0x46e7884e8a8d6663135d36a80ab0a9ee2bd286e6f82e4a359c7a2a7841224298""",701
"""0x394937bc18f92f976faba76bf816b9da53d3708d14c71423ec490bbb7820b627""",700
"""0x0fab831e2808129816a241472e64f9b680d6035326e696391c28acea4eb908ba""",700
"""0xc92a2a570ee0048da8779a75ac2c8b98d1d0af49683aa92e46bb844899be1dc9""",700
"""0x00a1f4b87db8f9e16f3912a8099580a6293805a95fb45f0d8d8ab50b685a8daa""",700
"""0xeb5b8ead45667b9d1e0fbbf0eefec4122235afacee05aace04ca71b9b2f501ed""",700
"""0x809ffb6ef139520603e4a5d620a9e501e5a885486bb3e6b1ddc8b49fe759cbfe""",700
"""0x044eae45065067a8dc42280ed50bbbf00537bc8a68de2e516e0206a28bab1878""",700


### Which contracts have the most leading zeros?

In [20]:
sorted_contracts = (
    pl.scan_parquet(data_path)
    .select(pl.col('contract_address'))
    .sort('contract_address')
    .unique()
    .head(10)
    .collect(streaming=True)
)['contract_address']

for binary_address in sorted_contracts:
    address = ctc.binary_convert(binary_address, 'prefix_hex')
    print(address, 40 - len(address.lstrip('0x')))

0x000000000000005394b1ed7aa43cc33358517d24 14
0x000000000000006f6502b7f2bbac8c30a3f67e9a 14
0x00000000000001ad428e4906ae43d8f9852d0dd6 13
0x00000000000003441d59dde9a90bffb1cd3fabf1 13
0x00000000000003c98e6a44a835cb776385bed207 13
0x00000000000006c7676171937c444f6bde3d6282 13
0x000000000000073760fc462304360d9e887e4ef4 13
0x000000000000084e91743124a982076c59f10084 13
0x000000000000093dfd365e70a598de99c41ec840 13
0x0000000000000ad24e80fd803c6ac37206a45f15 13


# We can also perform targeted queries

### Who deployed this contract? When? What was the creation transaction?

In [21]:
DAI = '0x6b175474e89094c44da98b954eedeac495271d0f'

In [22]:
binary_address = ctc.binary_convert(DAI, 'binary')

result = (
    pl.scan_parquet(data_path)
    .filter(pl.col('contract_address') == binary_address)
    .collect(streaming=True)
)
pl_utils.binary_columns_to_prefix_hex(result)

STREAMING CHUNK SIZE: 40000 rows


block_number,create_index,transaction_hash,contract_address,deployer,factory,init_code,code,init_code_hash,code_hash
i64,i64,str,str,str,str,str,str,str,str
8928158,0,"""0x495402df7d45fe36329b0bd94487f49baee62026d50f654600f6771bd2a596ab""","""0x6b175474e89094c44da98b954eedeac495271d0f""","""0xddb108893104de4e1c6d0e47c42237db4e617acc""","""0xb5b06a16621616875a6c2637948bf98ea57c58fa""","""0x608060405234801561001057600080fd5b506040516120d33803806120d38339818...","""0x608060405234801561001057600080fd5b50600436106101425760003560e01c806...","""0xa24ab178906449d0fd98af1310988221249de763de7f661bce0b613010689770""","""0x4e36f96ee1667a663dfaac57c4d185a0e369a3a217e0079d49620f34f85d1ac7"""


### What are all of the deployed Uniswap V2 pools?

In [23]:
example_pool = '0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc'

In [24]:
example_bytecode = await ctc.rpc.async_eth_get_code(example_pool)
example_code_hash = ctc.keccak(example_bytecode, output_format='binary')

result = (
    pl.scan_parquet(data_path)
    .filter(pl.col('code_hash') == example_code_hash)
    .collect(streaming=True)
)
pl_utils.binary_columns_to_prefix_hex(result)

STREAMING CHUNK SIZE: 40000 rows


block_number,create_index,transaction_hash,contract_address,deployer,factory,init_code,code,init_code_hash,code_hash
i64,i64,str,str,str,str,str,str,str,str
10008355,14,"""0xd07cbde817318492092cc7a27b3064a69bd893c01cb593d6029683ffd290ab3a""","""0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc""","""0xf94e5cdf41247e268d4847c30a0dc2893b33e85d""","""0x5c69bee701ef814a2b6a3edd4b1652cb9cc5aa6f""","""0x60806040526001600c5534801561001557600080fd5b506040514690806052612d2...","""0x608060405234801561001057600080fd5b50600436106101b95760003560e01c806...","""0x96e8ac4277198ff8b6f785478aa9a39f403cb768dd02cbee326c3e7da348845f""","""0x5b83bdbcc56b2e630f2807bbadd2b0c21619108066b92a58de081261089e9ce5"""
10008500,0,"""0xb0621ca74cee9f540dda6d575f6a7b876133b42684c1259aaeb59c831410ccb2""","""0x3139ffc91b99aa94da8a2dc13f1fc36f9bdc98ee""","""0x3a5a935b58901c25234fc481574bb7fde5135393""","""0x5c69bee701ef814a2b6a3edd4b1652cb9cc5aa6f""","""0x60806040526001600c5534801561001557600080fd5b506040514690806052612d2...","""0x608060405234801561001057600080fd5b50600436106101b95760003560e01c806...","""0x96e8ac4277198ff8b6f785478aa9a39f403cb768dd02cbee326c3e7da348845f""","""0x5b83bdbcc56b2e630f2807bbadd2b0c21619108066b92a58de081261089e9ce5"""
10014178,4,"""0x98f92a0b6063b0b581722cecf8ef01131c24c4377e6d3323d9184d1a262fc168""","""0x12ede161c702d1494612d19f05992f43aa6a26fb""","""0x8688a84fcfd84d8f78020d0fc0b35987cc58911f""","""0x5c69bee701ef814a2b6a3edd4b1652cb9cc5aa6f""","""0x60806040526001600c5534801561001557600080fd5b506040514690806052612d2...","""0x608060405234801561001057600080fd5b50600436106101b95760003560e01c806...","""0x96e8ac4277198ff8b6f785478aa9a39f403cb768dd02cbee326c3e7da348845f""","""0x5b83bdbcc56b2e630f2807bbadd2b0c21619108066b92a58de081261089e9ce5"""
10042267,0,"""0xc4c1840db940f5075c5404266efe50d7c65bc079f318ab8e5a10b8a432b49d30""","""0xa478c2975ab1ea89e8196811f51a7b7ade33eb11""","""0x74aa01d162e6dc6a657cac857418c403d48e2d77""","""0x5c69bee701ef814a2b6a3edd4b1652cb9cc5aa6f""","""0x60806040526001600c5534801561001557600080fd5b506040514690806052612d2...","""0x608060405234801561001057600080fd5b50600436106101b95760003560e01c806...","""0x96e8ac4277198ff8b6f785478aa9a39f403cb768dd02cbee326c3e7da348845f""","""0x5b83bdbcc56b2e630f2807bbadd2b0c21619108066b92a58de081261089e9ce5"""
10060679,0,"""0xee29b2a9306dab868b53919836c4a81feca435de6902c230c291b576ec9fa680""","""0x07f068ca326a469fc1d87d85d448990c8cba7df9""","""0xb96abc18d24eeda781f83be55ed737056965944b""","""0x5c69bee701ef814a2b6a3edd4b1652cb9cc5aa6f""","""0x60806040526001600c5534801561001557600080fd5b506040514690806052612d2...","""0x608060405234801561001057600080fd5b50600436106101b95760003560e01c806...","""0x96e8ac4277198ff8b6f785478aa9a39f403cb768dd02cbee326c3e7da348845f""","""0x5b83bdbcc56b2e630f2807bbadd2b0c21619108066b92a58de081261089e9ce5"""
10060832,0,"""0x966e4e9acaac28e44aafc937918718bc66804a3c94ac656e5e57faedd314c947""","""0xae461ca67b15dc8dc81ce7615e0320da1a9ab8d5""","""0x8688a84fcfd84d8f78020d0fc0b35987cc58911f""","""0x5c69bee701ef814a2b6a3edd4b1652cb9cc5aa6f""","""0x60806040526001600c5534801561001557600080fd5b506040514690806052612d2...","""0x608060405234801561001057600080fd5b50600436106101b95760003560e01c806...","""0x96e8ac4277198ff8b6f785478aa9a39f403cb768dd02cbee326c3e7da348845f""","""0x5b83bdbcc56b2e630f2807bbadd2b0c21619108066b92a58de081261089e9ce5"""
10061003,1,"""0x2e150ec518660ada856c2c57689bb45c5600bdbd38fc58043d99e99d7a031f83""","""0xce407cd7b95b39d3b4d53065e711e713dd5c5999""","""0x11e4857bb9993a50c685a79afad4e6f65d518dda""","""0x5c69bee701ef814a2b6a3edd4b1652cb9cc5aa6f""","""0x60806040526001600c5534801561001557600080fd5b506040514690806052612d2...","""0x608060405234801561001057600080fd5b50600436106101b95760003560e01c806...","""0x96e8ac4277198ff8b6f785478aa9a39f403cb768dd02cbee326c3e7da348845f""","""0x5b83bdbcc56b2e630f2807bbadd2b0c21619108066b92a58de081261089e9ce5"""
10061271,0,"""0x9dca7318a5ebb0a585bbb5749f1d4d4dd1aa93bfae896cd53ba50cf1693d22e0""","""0x33c2d48bc95fb7d0199c5c693e7a9f527145